In [11]:
# Imports
import yaml
import json
import requests
import re
from pprint import pp


In [12]:
x = yaml.safe_load(open(r"C:\Users\srappel\Documents\GitHub\GeoDiscovery-Utils\opendataharvest\OpenDataSites.yaml", "r"))

# print(x["Sites"].keys())

catalog_list = []

In [13]:
for site in x["TestSites"].keys():
    r = requests.get(x["TestSites"][site]["SiteURL"])
    if r.status_code != 200:
        break
    else:
        try:
            catalog_json = json.loads(r.content)
        except JSONDecodeError:
            print(f"The content from {site} is not a valid JSON document.")
            continue

        catalog_list.append(catalog_json)

In [14]:
for catalog in catalog_list:
    for dataset in catalog["dataset"]:
        pp(dataset)
        print(dataset['title'])
        print()

{'@type': 'dcat:Dataset',
 'identifier': 'https://www.arcgis.com/home/item.html?id=2347970abf3f4d8593ad30bd11719af3',
 'landingPage': 'https://gis-mclio.opendata.arcgis.com/datasets/MCLIO::2020-aerial-imagery-1',
 'title': '2020 Aerial Imagery',
 'description': "<div style='text-align:Left;'><p><span>3-inch ortho-rectified "
                'vertical aerial photography covering Milwaukee County '
                'Wisconsin - captured April 2020.</span></p><p><span>The '
                'dataset consists of tiled orthogonal imagery produced from \n'
                'nadir images captured by Pictometry International. Automatic '
                'aerial \n'
                'triangulation (AAT) was performed. The triangulated frames '
                'were rectified to a \n'
                'LiDAR derived DEM. Mosaicking was performed using an '
                'automatic seaming algorithm \n'
                'and manually edited to eliminate seams through elevated '
                'featu

In [15]:
test_dataset = catalog['dataset'][0]
test_dataset

{'@type': 'dcat:Dataset',
 'identifier': 'https://www.arcgis.com/home/item.html?id=2347970abf3f4d8593ad30bd11719af3',
 'landingPage': 'https://gis-mclio.opendata.arcgis.com/datasets/MCLIO::2020-aerial-imagery-1',
 'title': '2020 Aerial Imagery',
 'description': "<div style='text-align:Left;'><p><span>3-inch ortho-rectified vertical aerial photography covering Milwaukee County Wisconsin - captured April 2020.</span></p><p><span>The dataset consists of tiled orthogonal imagery produced from \nnadir images captured by Pictometry International. Automatic aerial \ntriangulation (AAT) was performed. The triangulated frames were rectified to a \nLiDAR derived DEM. Mosaicking was performed using an automatic seaming algorithm \nand manually edited to eliminate seams through elevated features where \npossible.</span></p></div>",
 'keyword': ['aerial', 'imagery', '2020', 'Milwaukee County'],
 'issued': '2020-10-12T15:47:43.000Z',
 'modified': '2023-12-27T19:02:13.000Z',
 'publisher': {'name': 'M

In [22]:
class aardvark:

    ## TODO: Mint an ark if one doesnt already exist.
    ## TODO: If ARK does exist, pull it
    ## TODO: Make updates to NOID metadata if needed
    ## TODO: MAYBE WE JUST USE THEIR ID e.g. `MCLIO::milwaukee-county-trails` as long as the colons don't mess things up.

    def __init__(self, dataset_dict):
        # Hard Coded
        self.gbl_resourceClass_sm: list = ["Datasets"] # Required
        self.dct_accessRights_s: str = "public"# Required
        self.gbl_mdModified_dt: str # datetime.now() formatted # Required
        self.gbl_mdVersion_s: str = "Aardvark" # Required
        self.dct_language_sm: list = "English"
        self.schema_provider_s: str = "American Geographical Society Library – UWM Libraries"
        self.gbl_suppressed_b: bool = False

        # dct_title_s (REQUIRED)
        if not 'title' in dataset_dict:
            raise error("title is required")
        else:
            self.dct_title_s: str = dataset_dict['title']

        assert self.dct_title_s.__class__ == str
        assert len(self.dct_title_s) > 0


        # dct_description_sm
        if 'description' in dataset_dict:
            html_strip = re.sub('<[^<]+?>', '', dataset_dict['description']) # Strips HTML
            self.dct_description_sm: list = [html_strip]

        assert self.dct_description_sm is not None


        # dct_creator_sm
        if "publisher" in dataset_dict:
            if "name" in dataset_dict['publisher']:
                self.dct_creator_sm = [dataset_dict['publisher']['name']]

        assert self.dct_creator_sm is not None
            

        # dct_issued_s
        if "issued" in dataset_dict:
            self.dct_issued_s = dataset_dict['issued']
            
        assert self.dct_issued_s is not None

        # locn_geometry & dcat_bbox
        # Assumes a polygon bounding box
        if 'spatial' in dataset_dict:
            coordlist = spatial.split(',')
            bbox = ''
            for coord in coordlist:
                bbox = bbox + coord + ','

            bbox = bbox.rstrip(",")
            self.locn_geometry = f"ENVELOPE({bbox})"
            self.dcat_bbox = f"ENVELOPE({bbox})"


        # dcat_keyword_sm (string multiple!)
        if 'keyword' in dataset_dict:
            self.dcat_keyword_sm = json.dumps(dataset_dict['keyword'])



    # Variable

    
    md_id: str # id is a built in function! # Required
    gbl_resourceType_sm: list
    dcat_theme_sm: list
    dct_temporal_sm: list
    gbl_indexYear_im: list
    dct_spatial_sm: list
    dct_rights_sm: list
    dct_format_s: str
    dct_references_s: str
    dct_identifier_sm: list
    dct_alternative_sm: list
    pcdm_memberOf_sm: list

    

  


In [23]:
test_aardvark_object = aardvark(test_dataset)
print(test_aardvark_object.dct_title_s)
print(test_aardvark_object.dct_description_sm)
print(test_aardvark_object.dct_creator_sm)
print(test_aardvark_object.dct_issued_s)
print(test_aardvark_object.locn_geometry)
print(test_aardvark_object.dcat_bbox)
print(test_aardvark_object.dcat_keyword_sm)


2020 Aerial Imagery
['3-inch ortho-rectified vertical aerial photography covering Milwaukee County Wisconsin - captured April 2020.The dataset consists of tiled orthogonal imagery produced from \nnadir images captured by Pictometry International. Automatic aerial \ntriangulation (AAT) was performed. The triangulated frames were rectified to a \nLiDAR derived DEM. Mosaicking was performed using an automatic seaming algorithm \nand manually edited to eliminate seams through elevated features where \npossible.']
['Milwaukee County GIS & Land Information']
2020-10-12T15:47:43.000Z
ENVELOPE(-88.1116,42.8348,-87.8001,43.1997)
ENVELOPE(-88.1116,42.8348,-87.8001,43.1997)
["aerial", "imagery", "2020", "Milwaukee County"]
